In [44]:
# load libraries & ignore warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")

In [45]:
# Load and preprocess data from "sba24130_DVT_Summer_CA.ipynb"
df = pd.read_csv("fremont_df.csv")
df.columns = ['Date', 'Total', 'West', 'East']
df = df.dropna()
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Hour'] = df['Date'].dt.hour
df['Weekday'] = df['Date'].dt.day_name()
df['Weekend'] = df['Weekday'].isin(['Saturday', 'Sunday'])

In [46]:
# Define UI widgets

In [47]:
# Slider for year
year_slider = widgets.IntRangeSlider(
    value=[2012, 2025], min=2012, max=2025,
    description='Year Range:', continuous_update=False, layout=widgets.Layout(width='95%')
)

# Weekday / weekend filter
traffic_toggle = widgets.ToggleButtons(
    options=['All', 'Weekday', 'Weekend'],
    description='Traffic Type:',
    button_style='info'
)

# Direction of traffic filter
direction_dropdown = widgets.Dropdown(
    options=['Total', 'East', 'West'],
    value='Total',
    description='Direction:'
)

out = widgets.Output()

In [48]:
# Define consistent color palette for dashboard
color_main = "#2a9d8f"        # Total
color_east = "#2a9d8f"        # East
color_west = "#e76f51"        # West
color_secondary = "#e76f51"   # Weekend / contrast

In [49]:
# --- Define dashboard logic ---
def update_dashboard(year_range, traffic_type, direction):
    with out:
        clear_output(wait=True)
        filtered_df = df[(df['Year'] >= year_range[0]) & (df['Year'] <= year_range[1])]
        if traffic_type == 'Weekday':
            filtered_df = filtered_df[filtered_df['Weekend'] == False]
        elif traffic_type == 'Weekend':
            filtered_df = filtered_df[filtered_df['Weekend'] == True]

        if direction == 'East':
            filtered_df['Total'] = filtered_df['East']
            direction_label = "East"
        elif direction == 'West':
            filtered_df['Total'] = filtered_df['West']
            direction_label = "West"
        else:
            direction_label = "Total"

        fig, axs = plt.subplots(3, 2, figsize=(18, 12))

        # Monthly Bicycle Traffic Over Time
        monthly_data = filtered_df.resample('M', on='Date')['Total'].sum()
        axs[0, 0].plot(monthly_data.index, monthly_data.values, color=color_main)
        axs[0, 0].set_title(f"Monthly Bicycle Traffic Over Time ({direction_label})")
        axs[0, 0].set_xlabel("Month")
        axs[0, 0].set_ylabel("Total Bicycle Count")

        # Total Bicycle Traffic by Day of Week
        weekly_totals = filtered_df.groupby('Weekday')['Total'].sum()
        week_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        weekly_totals = weekly_totals.reindex(week_order)
        axs[0, 1].bar(weekly_totals.index, weekly_totals.values, color=color_main)
        axs[0, 1].set_title(f"Total Bicycle Traffic by Day of the Week ({direction_label})")
        axs[0, 1].set_xlabel("Day of Week")
        axs[0, 1].set_ylabel("Total Bicycle Count")

        # Average Hourly Bicycle Traffic (East vs West)
        if direction == 'Total':
            east_hourly = filtered_df.groupby('Hour')['East'].mean()
            west_hourly = filtered_df.groupby('Hour')['West'].mean()
            axs[1, 0].plot(east_hourly.index, east_hourly.values, label='East', color=color_east)
            axs[1, 0].plot(west_hourly.index, west_hourly.values, label='West', color=color_west)
            axs[1, 0].legend()
        else:
            axs[1, 0].plot(filtered_df.groupby('Hour')['Total'].mean(), color=color_main, label=direction_label)
            axs[1, 0].legend()
        axs[1, 0].set_title("Average Hourly Bicycle Traffic")
        axs[1, 0].set_xlabel("Hour of Day")
        axs[1, 0].set_ylabel("Average Bicycle Count")

        # Average Hourly Bicycle Traffic: Weekday vs Weekend
        avg_ww = filtered_df.groupby(['Weekend', 'Hour'])['Total'].mean().unstack(level=0)
        if False in avg_ww.columns:
            axs[1, 1].plot(avg_ww.index, avg_ww[False], label='Weekday', color=color_main)
        if True in avg_ww.columns:
            axs[1, 1].plot(avg_ww.index, avg_ww[True], label='Weekend', color=color_secondary)
        axs[1, 1].legend()
        axs[1, 1].set_title("Average Hourly Bicycle Traffic: Weekday vs Weekend")
        axs[1, 1].set_xlabel("Hour of Day")
        axs[1, 1].set_ylabel("Average Bicycle Count")

        # Total Bicycle Counts by Year
        yearly_totals = filtered_df.groupby('Year')['Total'].sum()
        axs[2, 0].bar(yearly_totals.index, yearly_totals.values, color= color_main)
        axs[2, 0].set_title(f"Total Bicycle Counts by Year ({direction_label})")
        axs[2, 0].set_xlabel("Year")
        axs[2, 0].set_ylabel("Total Bicycle Count")

        # Average Bicycle Count by Hour and Weekday (Heatmap)
        heat_data = filtered_df.groupby(['Weekday', 'Hour'])['Total'].mean().unstack()
        heat_data = heat_data.reindex(week_order)
        sns.heatmap(heat_data, ax=axs[2, 1], cmap="YlGnBu")
        axs[2, 1].set_title("Average Bicycle Count by Hour and Weekday")
        axs[2, 1].set_xlabel("Hour of Day")
        axs[2, 1].set_ylabel("Day of Week")

        plt.tight_layout()
        plt.show()

In [50]:
# Connect widgets
ui = widgets.VBox([year_slider, traffic_toggle, direction_dropdown])
year_slider.observe(lambda change: update_dashboard(year_slider.value, traffic_toggle.value, direction_dropdown.value), names='value')
traffic_toggle.observe(lambda change: update_dashboard(year_slider.value, traffic_toggle.value, direction_dropdown.value), names='value')
direction_dropdown.observe(lambda change: update_dashboard(year_slider.value, traffic_toggle.value, direction_dropdown.value), names='value')

## Fremont Bicylce Data Dashboard 

In [52]:
# Initial Display
display(ui, out)
update_dashboard(year_slider.value, traffic_toggle.value, direction_dropdown.value)

Output()